# DFT Course - 2022-23
## Beatriz Helena Cogollo-Olivo

## Tutorial 01 - Convergence testing

In order to use our computer efficiently, it is crucial to find a setting of parameters that balances the accuracy and reliability of the simulation, and the computational resources. 

There are many convergence test methods. Here we will exlplore the energy convergence against the cutoff energy, the k-points, and the smearing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Convergence with cutoff energy for silicon bulk

$ecut$ is the keyword to set the number of plane waves used to calculate the DFT. If the value is high for the calculation of plane waves, the accuracy of the calculation is improved, but it takes a longer time to calculate it.

In [ ]:
#Let's download the pseudopotential
!wget http://pseudopotentials.quantum-espresso.org/upf_files/Si.pz-vbc.UPF

In [ ]:
%%writefile ecut-convergence.sh

#!/bin/sh
NAME="ecut"
for CUTOFF in AA BB CC DD EE FF #The ecut values are defined in this list
do
cat > ${NAME}_${CUTOFF}.in << EOF
 &control
    calculation = 'scf',
    prefix = 'silicon'
    outdir = './tmp/'
    pseudo_dir = './'
 /
 &system
    ibrav =  2,
    celldm(1) = 10.0,
    nat =  2,
    ntyp = 1,
    ecutwfc = $CUTOFF
 /
 &electrons
    mixing_beta = 0.6
 /
ATOMIC_SPECIES
 Si 28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS (alat)
 Si 0.0 0.0 0.0
 Si 0.25 0.25 0.25
K_POINTS (automatic)
  6 6 6 1 1 1
EOF

pw.x < ${NAME}_${CUTOFF}.in > ${NAME}_${CUTOFF}.out
echo ${NAME}_${CUTOFF}
grep ! ${NAME}_${CUTOFF}.out

done

In [ ]:
!chmod +x ecut-convergence.sh
!sh ecut-convergence.sh

In [ ]:
x1 = [] #Include the ecut values separated by commas
y1 = [] #Include the total energy values separated by commas
plt.xlabel('Cutoff energy (Ry)')
plt.ylabel('Total energy (Ry)')
plt.plot(x1, y1)
plt.show()

In order to be certain of the number of plane wave requiered for an accurate simulation, we are looking for energy differences lower than 5 meV/atom.

In [ ]:
new_y1 = [(i+ENERGY)*(13.6056980659*1000/NAT) for i in y1] #Sum the lowest energy value and the number of atoms per cell 
plt.xlabel('Cutoff energy (Ry)')
plt.ylabel('Total energy difference (meV/atom)')
plt.ylim(0, LIM) #The y-axis limits can be changed here
plt.plot(x1, new_y1)
plt.grid()
plt.show()

#### Question 1
What is the lowest value for the cutoff energy that ensures an energy difference lower than 5 meV/atom?

### Convergence with $k$ points for silicon bulk

We can run similar convergence test against other parameter, and choose the best value of that particular parameter. Here we will try to calculate number of k-points in the Monkhorst-Pack mesh.

In [ ]:
%%writefile kpts-convergence.sh

#!/bin/sh
NAME="kpts"
for K in AA BB CC DD #The k points values are defined in this list
do
cat > ${NAME}_${K}.in << EOF
 &control
    calculation = 'scf',
    prefix = 'silicon'
    outdir = './tmp/'
    pseudo_dir = './'
 /
 &system
    ibrav =  2,
    celldm(1) = 10.0,
    nat =  2,
    ntyp = 1,
    ecutwfc = EE !#Set the selected ecut value
 /
 &electrons
    mixing_beta = 0.6
 /
ATOMIC_SPECIES
 Si 28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS (alat)
 Si 0.0 0.0 0.0
 Si 0.25 0.25 0.25
K_POINTS (automatic)
  $K $K $K 1 1 1
EOF

pw.x < ${NAME}_${K}.in > ${NAME}_${K}.out
echo ${NAME}_${K}
grep ! ${NAME}_${K}.out

done

In [ ]:
!chmod +x kpts-convergence.sh
!sh kpts-convergence.sh

In [ ]:
x2 = [] #Include the ecut values separated by commas
y2 = [] #Include the total energy values separated by commas
plt.xlabel('K points grid')
plt.ylabel('Total energy (Ry)')
plt.plot(x2, y2)
plt.show()

In [ ]:
new_y2 = [(i+ENERGY)*(13.6056980659*1000/NAT) for i in y2] #Sum the lowest energy value and the number of atoms per cell 
plt.xlabel('K points grid')
plt.ylabel('Total energy difference (meV/atom)')
plt.ylim(0, LIM) #The y-axis limits can be changed here
plt.plot(x2, new_y2)
plt.grid()
plt.show()

#### Question 2
What is the lowest $k$ point mesh that ensures an energy difference lower than 5 meV/atom?

### Convergence with smearing for aluminum bulk

$smearing$ is a mathematical trick implemented Quantum ESPRESSO to help in convergence for metallic systems.

In [ ]:
#Let's download the pseudopotential
!wget https://pseudopotentials.quantum-espresso.org/upf_files/Al.pbe-n-rrkjus_psl.1.0.0.UPF

In [ ]:
%%writefile smearing-convergence.sh

#!/bin/sh
NAME="smearing"
for DEGAUSS in AA BB CC DD EE #The degauss values are defined in this list
do
cat > ${NAME}_${DEGAUSS}.in << EOF
&CONTROL 
  calculation= 'scf', 
  restart_mode= 'from_scratch', 
  prefix= 'al', 
  outdir= './tmp/', 
  pseudo_dir= './' 
/ 
&SYSTEM 
  ibrav= 2, 
  celldm(1) = 7.63075, 
  nat= 1, 
  ntyp= 1, 
  ecutwfc= 50, 
  ecutrho= 500, 
  occupations= 'smearing',
  smearing= 'gaussian', !#The type of broadening can be changed here
  degauss= $DEGAUSS
/ 
&ELECTRONS
  conv_thr= 1e-8
/
ATOMIC_SPECIES
  Al 26.981539 Al.pbe-n-rrkjus_psl.1.0.0.UPF
ATOMIC_POSITIONS (alat)
  Al 0.00 0.00 0.00
K_POINTS (automatic)
  14 14 14 0 0 0
EOF

pw.x < ${NAME}_${DEGAUSS}.in > ${NAME}_${DEGAUSS}.out
echo ${NAME}_${DEGAUSS}
grep ! ${NAME}_${DEGAUSS}.out

done

In [ ]:
!chmod +x smearing-convergence.sh
!sh smearing-convergence.sh

In [ ]:
x3 = [] #Include the degauss values separated by commas
gauss = [] #Include the total energy values separated by commas
plt.xlabel('Degauss (Ry)')
plt.ylabel('Total energy (Ry)')
plt.plot(x3, gauss)
plt.show()

#### Question 3
Change the type of broadening from Gaussian ($gaussian$), to Methfessel-Paxton ($m-p$), Marzari-Vanderbilt ($m-v$), and Fermi-Dirac ($f-d$), for the same smearing values defined above and plot the calculated energies in the following instruction:

In [ ]:
x3 = [] #Include the degauss values separated by commas
gauss = [] #Include the total energy values separated by commas
mp = []
mv = []
fd = []
plt.xlabel('Degauss (Ry)')
plt.ylabel('Total energy (Ry)')
plt.plot(x3, gauss, label='Gauss')
plt.plot(x3, mp, label='Methfessel-Paxton')
plt.plot(x3, mv, label='Marzari-Vanderbilt')
plt.plot(x3, fd, label='Fermi-Dirac')
plt.legend()
plt.show()

#### Question 4
Which broadening(s) is(are) less depending on the degauss value?

#### Question 5
With the selected broadening(s), for the same smearing values defined above, plot the calculated energies by varying the k points grid with these values: 10, 12, 14, 16

In [ ]:
x3 = [] #Include the degauss values separated by commas
B1_10 = [] #Include the total energy values separated by commas
B1_12 = []
B1_14 = []
B1_16 = []
plt.plot(x3, B1_10, label='B1_10') #In label write the selected broadening
plt.plot(x3, B1_12, label='B1_12')
plt.plot(x3, B1_14, label='B1_14')
plt.plot(x3, B1_16, label='B1_16')
plt.xlabel('Degauss (Ry)')
plt.ylabel('Total energy (Ry)')
plt.legend()
plt.show()

In [ ]:
x3 = [0.02, 0.04, 0.06, 0.08, 0.10] #Include the degauss values separated by commas
B2_10 = [] #Include the total energy values separated by commas
B2_12 = []
B2_14 = []
B2_16 = []
plt.plot(x3, B2_10, label='B2_10') #In label write the selected broadening
plt.plot(x3, B2_12, label='B2_12')
plt.plot(x3, B2_14, label='B2_14')
plt.plot(x3, B2_16, label='B2_16')
plt.xlabel('Degauss (Ry)')
plt.ylabel('Total energy (Ry)')
plt.legend()
plt.show()

#### Question 6
Between the selected broadening(s), which one converges the best?

#### Question 7
What is the best $k$ points and smearing arragement for aluminum?